# Labsheet 5: Finite State Machine

By this stage we are beginning to collect a variety of functions and behaviours for our 3Pi+ robot. Looking ahead at the line following challenge, we know that the robot will need to:
1. Join the Line
2. Follow the Line
3. Navigate Corners
4. Navigate an Intersection / Traverse a Gap in the Line
5. Determine the end of the Line
6. Return to Home (start position)

The complexity of our control software is growing - meaning that, there are many pathways of execution that your robot might take depending on the perceived environmental conditions.  The general layout and design of your code can quite messy and complicated.  This is quite normal in a development cycle, but it can be a major cause of time spent solving bugs.  It is good practice to consolidate your code and to reduce functional repetition.  

This labsheet will cover how to organise different behaviours for your robot, and to do so in a consistent style of coding.  When this is achieved, it is then much easier to enable or disable behaviours, to change the mechanisms to start or end behaviours, or to change the sequential transitions between behaviours.

In previous labsheets we have:

- Familiarised with the Arduino IDE, Sketch files, uploading to the 3Pi+, and utilising the Serial interface
- Implemented basic operation of the 3Pi+ motors, and encapsulated this within safe and confident function(s), and utilised a `class`.
- Implemented basic read functionality of the 3 central IR Line sensors facing the work surface, and encapsulated this within function(s).
- Explored the use of `millis()` to approximate task-scheduling on the 3Pi+.

We have also developed behaviour `primitives`:
- operating the motors to drive forwards/backwards, turning on the spot, moving in an arc.
- combining the operation of the line sensors and motors for line following, either with bang-bang or a weighted-measurement technique.
- made early explorations of how well the robot can join a line from different starting locations and angles.

**It is strongly recommended that before you attempt this worksheet, you go back and tidy up your code.  You may even want to create some clean and simple programs for sub-tasks or sub-behaviours (e.g., "Drive straight", "follow line").  It will then be easier to add these parts to the exercises in this labsheet.**

For this labsheet it can also help to do some sketching on paper to help organise the complexity of the task.


<hr><br><br><br><br>

# Finite State Machines

<a href="https://en.wikipedia.org/wiki/Finite-state_machine">Wikipedia</a> defines a Finite State Machine (FSM) as:


- _"A finite-state machine (FSM) ... is a mathematical model of computation. It is an abstract machine that can be in exactly one of a finite number of states at any given time. The FSM can change from one state to another in response to some external inputs; the change from one state to another is called a transition. An FSM is defined by a list of its states, its initial state, and the conditions for each transition."_

The following example, also from Wikipedia, will also help us to understand how to implement a FSM for our robot.

An example of a simple mechanism that can be modeled by a state machine is a turnstile.

A turnstile, used to control access to subways and amusement park rides, is a gate with three rotating arms at waist height, one across the entryway. Initially the arms are locked, blocking the entry, preventing patrons from passing through. Depositing a coin or token in a slot on the turnstile unlocks the arms, allowing a single customer to push through. After the customer passes through, the arms are locked again until another coin is inserted.

Considered as a state machine, the turnstile has **two possible states: Locked and Unlocked**. There are **two possible inputs that affect its state: putting a coin in the slot (coin) and pushing the arm (push)**. In the locked state, pushing on the arm has no effect; no matter how many times the input push is given, it stays in the locked state. Putting a coin in – that is, giving the machine a coin input – shifts the state from Locked to Unlocked. In the unlocked state, putting additional coins in has no effect; that is, giving additional coin inputs does not change the state. However, a customer pushing through the arms, giving a push input, shifts the state back to Locked.

The turnstile state machine can be represented by a state transition table, showing for each possible state, the transitions between them (based upon the inputs given to the machine) and the outputs resulting from each input:

<img src="https://github.com/paulodowd/EMATM0054_53_23-24/blob/main/Images/Turnstile_state_machine_colored.png?raw=true"></br>

In the above diagram we can see that:
- The system starts at the Black dot, and therefore immediately moves to an **initial state** of **locked**
- Sensing **push** while **locked** keeps the system in state **locked**.
- Sensing a **coin** while in **locked** transitions to state **unlocked**.
- Sensing a **coin** while in **unlocked** keeps the system in state **unlocked**.
- Sensing **push** while in **unlocked** transitions the state to **locked**.
- The FSM will never go to an undetermined state.










# Developing a FSM

Building a FSM can become very complicated.  It is best to get something simple working first. To start with, we will design a FSM to get our robot to drive forward from the starting box on the map, and then stop when it finds the line.  

When designing an FSM, we need to step away from our human understanding of how the robot should behave (<a href="https://en.wikipedia.org/wiki/Anthropomorphism">anthropomorphism</a>).  Instead, it is better to attempt to anticipate how the code you have written will operate.

We need to decompose the desired overall behaviour into smaller behaviours which rely only on the information available to the robot.  We need to make sure we catch any assumptions about the robot, any ambiguity, and any potentially ambiguous circumstances.  

For example, what should your robot do as soon as you turn it on?  The robot is fundamentally not-intelligent, so we need to embed our intelligence into the robot with code.

In our case, it would be useful if, when we turn on the robot, it makes sure:
1. all motors are off so it doesn't run away.
2. it signals to you (light or sound) for some time so you know it is safe to to locate the robot on the surface.
3. it then performs any calibration protocols from a safe starting position.
4. it returns actuators and sensors to a known initial state, and outputs a final output signal to you (light or sound) to indicate it is ready.

This short behaviour would operate within our **initial state**.  After this, the robot would need to `transition` into a new state of operation, employing further behaviours.

To start developing a FSM, we will work on moving between **3 states**:

1. An **initial** state: the sequence outlined above.  
2. A **drive forwards** state: to drive out of the starting box, and attempt to encounter the line, or otherwise `time out` the operation and signal an error.  
3. A **found line** state: to get the robot to stop when it finds the line, and to signal this success.

This is a good start to a FSM.  Hopefully you are already imagining how it could be extended.  To start, we can check that the robot is able to reliably move between just these three states appropriately.  Once this is complete, we can start to add further states to employ the appropriate functional behaviours.  





<hr><br><br><br><br>


# Exercise 1: Drawing a simple FSM on Paper (15minutes)

Please feel free to discuss these exercises with your peers and tutors:

1. In the above discussion, a one of the states was called `drive forwards`.  We could also name this as `join line`, since this is the desired activity we wish to observe.  Why might `drive forwards` and `join line` be usefully decomposed into two different states?

2. Draw a FSM for the above states.
    - Decide on some useful state names that you can use in code as status flags, such as "STATE_INITIALISE".
    - Annotate your FSM to include the `proprioceptive` and `exteroceptive` information used to inform when a `transition` is needed between `states`.
    - Remember to include elements such as a `time out` if the robot does not encounter a line whilst in the `driving forwards` state.
    - Make notes on your FSM to about pieces of code or techniques that you have developed which will be relevant.
    - If you have not used the on-board LEDs or audible buzzer, you may wish to investigate the built in examples.  These will be useful to signal the activity of the robot to you.
      - why might using the Serial Monitor be problematic?

    


<br><br><br><br>


# Exercise 2:  Code your 3 state FSM

You should now begin to code the 3 state FSM you drew on paper.  Start by coding the framework for your FSM, including:
- a global variable to track which state your system is in.
- `#define` or global variables to enumerate each possible state with a meaningful name.
- routines in `setup()` to initialise your global variables.
- an `if() {} elseif() {}` code block within `loop()` to select appropriate functions for each state.
- test functions associated to each state.


The following code extract provides a minimum framework for you to use.  
- Write in, or copy-and-paste in, code from your previous work, where it is relevant.  You will need all the setup routines for motors, encoders, line sensors, etc. Keep your code as tidy as possible, things will get harder to debug as you increase the system complexity.


- Save **versions** of your code - espcially if you get something working!  When something goes wrong, it is often useful to go back to the last best known working version.


- Remember that you can use **global variables** to make information available to all parts of your code.  However, global variables will keep their last assigned value, so you'll also have to remember to reset them if need be.

    
- Add **Serial.println()** at key points in your code to track of which state the robot is operating in.  You can remove these later.


- Remember to go back and look at labsheets if you have forgotten something.


```c
// Declare your different possible states here by enumerating them.
// Remember, a #define works like a find and replace in code.
#define STATE_INITIAL  0
#define STATE_         1
#define STATE_         2

// This will hold which state your robot is in. It will receive
// the numbers set above, but your code will be readable because
// strings used in the #define's
int state;


// put your setup code here, to run once:
void setup() {

  // Insert necessary initialising here.  e.g classes, pin modes
  // and states, Serial.begin()
  // ...
  
  // Set system state to intial  before robot begins to operate.
  state = STATE_INITIAL;
}


// put your main code here, to run repeatedly:
void loop() {

  // Always call a function to update the state
  // variable.  loop() should be returning quickly.
  // flow control is managed in updateState()
  updateState();

  // This is the basic structure for a FSM  Based on the value
  // of "state" variable, run appropriate code for robot behaviour.
  if( state == STATE_INITIAL ) {
              
           calibration();     
      
  } else if( state == STATE_DRIVE_FORWARDS ) {

           driveForwards();     
  
  } else if( state == STATE_FOUND_LINE ) {

           foundLineBeeps();
      
  } else {

          // You can catch situations where the robot
          // attempts to move into an unknown state.
          Serial.print("System Error, Unknown state: ");
          Serial.println( state );

          // You might want to develop codes to represent
          // what type of problem was encountered.
          Serial.print("Error code: ");
          Serial.println( error_code );

          // These functions might be useful to write.
          stopRobot();
          signalError();
  }

}

// A function to update the system variables, and to
// cause transitions between system states.
void updateState() {

}



// write this function to have your
// robot beep 5 times, across a total
// of 5 seconds.
void intialisingBeeps() {

}



// Write code that will command your
// robot to drive forwards straight
// until it detects the course line.
void driveForwards() {
  
}



// Write code that will deactivate all
// motors, and beep once for one second,
// then do nothing.
void foundLineBeep() {
  
}
```

# Exercise 3:  Build up your FSM

Now incrementally add behaviours to your FSM and ensure that transitions happen when you expect them to.  

- Break down the line following challenge into sub tasks.  Note that, this probably isn't just the different components of the line.  You may need small sub-behaviours between components of the line. You may even require small transition states within your code to set variables.   At this stage, consider adding the following states and behaviours to your robot:
  
- Add code so that your robot indicates a state change to you before moving on to the next system state.  This could be an LED or a beep.

- Return to paper based drawings of the FSM to keep track of how to write your code.  

- Remember that you can use *millis()* to create timestamps and check for elapsed time.  This might be useful to check if you have lost the line, or to wait without using delay().

- Experiment with stating your robot into different states, and disabling states.  Using a FSM can become very useful for debugging code.

- Consider adding a special `debug` state, which you can utilise for general debugging at any point.
    